In [1]:
from CustomPongEnv_v0_ram import Custom_Pong
import numpy as np
import random
random.seed(10)
np.random.seed(10)

env = Custom_Pong()
states=env.observation_space.shape[0]
actions=env.action_space.n

In [2]:
def state_to_obs(state, streward):
    
    factor = 5
    
    ball_x = state[0]
    o1_float = ball_x/factor
    o1 = int(o1_float)

    ball_y = state[1] 
    o2_float = ball_y/factor
    o2 = int(o2_float)

    ball_vx = state[2]
    o3_float = ball_vx
    o3_i = int(o3_float)
    o3 = 0 if o3_i==2 else 1 


    ball_vy = state[3]
    o4_float = ball_vx
    o4_i = int(o4_float)
    o4 = 0 if o4_i==2 else 1 

    paddle_pos = state[4]
    o5_float = paddle_pos/factor
    o5 = int(o5_float)

    paddle_vel = state[5]
    o6_float = paddle_vel
    o6_i = int(o6_float)
    o6 = 0 if o6_i==4 else 1 
    
    if(reward == -1):
        o7 = 0
    elif(reward == 1):
        o7 = 2
    else:
        o7 = 1
    
    observation = [o1, o2, o3, o4, o5, o6, o7]
    
    return(observation)

# Active Inference
## Setting up states and factors

In [4]:
import pymdp
from pymdp import utils
import numpy as np
from scipy.stats import dirichlet
from pymdp.agent import Agent

C:\Users\aswin\anaconda3\lib\site-packages\seaborn\rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\aswin\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [5]:
# (Hidden)Factors
s1_size = 42

num_states = [s1_size]
num_factors = len(num_states)

# Rewards
reward_modes = 3 #Max score-5 (assumption)

# Controls
s1_actions = ['Stay', 'Play-Up', 'Play-Down']

num_controls = [len(s1_actions)]

# Observations
o1_obs_size = 42
o2_obs_size = 42
o3_obs_size = 2
o4_obs_size = 2
o5_obs_size = 42
o6_obs_size = 2
reward_obs_size = reward_modes

num_obs = [o1_obs_size, o2_obs_size, o3_obs_size, o4_obs_size, o5_obs_size, o6_obs_size, reward_obs_size]
num_modalities = len(num_obs)

In [6]:
EPS_VAL = 1e-16 # Negligibleconstant

# Likelhiood Dynamics
a = utils.random_A_matrix(num_obs, num_states) * 0 + EPS_VAL
A = utils.random_A_matrix(num_obs, num_states)

# Transisition dynamics
# Initialised as random becuase the agent need to learn the dynamics

B = utils.random_B_matrix(num_states, num_controls)
b = utils.random_B_matrix(num_states, num_controls) * 0 + EPS_VAL
B[0].shape

# Normalising A and B as probability distributions

numS = num_states[0]
numA = num_controls[0]

for i in range(numS):
    A[0][:,i] = dirichlet.mean(a[0][:,i])
    A[1][:,i] = dirichlet.mean(a[1][:,i])
    A[2][:,i] = dirichlet.mean(a[2][:,i])
    A[3][:,i] = dirichlet.mean(a[3][:,i])
    A[4][:,i] = dirichlet.mean(a[4][:,i])
    A[5][:,i] = dirichlet.mean(a[5][:,i])
    A[6][:,i] = dirichlet.mean(a[6][:,i])
    for j in range(numA):
        B[0][:,i,j]=dirichlet.mean(b[0][:,i,j])

# Prior preferences for biasing the generative model to control behaviour

# The preferences are set uniform for all the hidden-states except the reward function
C = utils.obj_array_uniform(num_obs)

# Highest for the high-score and lowest for the lowscore
C_score = np.array([-5.8, 0 , 1])
# Normalising the prior preference
C[6] = pymdp.maths.softmax(1*C_score)
C[6]

D = utils.obj_array_uniform([s1_size])

In [7]:
# Dynamic programming active inference planner
from ai_agent_planner import action_dist

trials = 4
episodes = 120

In [8]:
# AI Agent-1 Planning horizon=5

In [9]:
%%time

planning_horizon = 5
T = planning_horizon

rally_length_raw = np.zeros((trials,episodes))

for trial in range(trials):
    print("trial", trial)
    
    EPS_VAL = 1e-16 #negligibleconstant
    # Likelhiood Dynamics
    a = utils.random_A_matrix(num_obs, num_states) * 0 + EPS_VAL
    A = utils.random_A_matrix(num_obs, num_states)

    # Transisition dynamics
    # Initialised as random becuase the agent need to learn the dynamics

    B = utils.random_B_matrix(num_states, num_controls)
    b = utils.random_B_matrix(num_states, num_controls) * 0 + EPS_VAL
    B[0].shape

    # Normalising A and B as probability distributions

    numS = num_states[0]
    numA = num_controls[0]

    for i in range(numS):
        A[0][:,i] = dirichlet.mean(a[0][:,i])
        A[1][:,i] = dirichlet.mean(a[1][:,i])
        A[2][:,i] = dirichlet.mean(a[2][:,i])
        A[3][:,i] = dirichlet.mean(a[3][:,i])
        A[4][:,i] = dirichlet.mean(a[4][:,i])
        A[5][:,i] = dirichlet.mean(a[5][:,i])
        A[6][:,i] = dirichlet.mean(a[6][:,i])
        for j in range(numA):
            B[0][:,i,j]=dirichlet.mean(b[0][:,i,j])

    # Prior preferences for biasing the generative model to control behaviour

    # The preferences are set uniform for all the hidden-states except the reward function
    C = utils.obj_array_uniform(num_obs)

    # Highest for the high-score and lowest for the lowscore
    C_score = np.array([-5.8, 0 , 1])
    # Normalising the prior preference
    C[6] = pymdp.maths.softmax(1*C_score)
    C[6]

    D = utils.obj_array_uniform([s1_size])
    
    for episode in range(episodes):
        # print("episode", episode)

        state = env.reset()
        done = False
        score = 0

        t=0
        while not done:
            # env.render()
            if(t==0):
                qs = D
                qs_prev = qs

            Q_pi = action_dist(A[6], B, C[6], T=planning_horizon, sm_par=1)
            action = np.random.choice([0,1,2], size=None, replace=True, p=np.matmul(Q_pi[t],qs[0]))

            state, reward, done, info = env.step(action)
            score += reward

            observation = state_to_obs(state, reward)

            qs = pymdp.inference.update_posterior_states(A, observation, prior = D)
            if(t == 0):
                qs0 = qs

            pA = A
            pB = B
            pD = D

            # Learning
            A = pymdp.learning.update_obs_likelihood_dirichlet(pA, A, observation, qs, lr=1.0, modalities='all')

            actions = np.array([int(action)])
            B = pymdp.learning.update_state_likelihood_dirichlet(pB, B, actions, qs, qs_prev, lr=1.0, factors='all')

            if(t == 0):
                D = pymdp.learning.update_state_prior_dirichlet(pD, qs0, lr=1.0, factors='all')
                D[0] = pymdp.maths.softmax(D[0])

            t += 1
            qs_prev = qs

            if(t == T-1):
                t = 0
                #Normalising A and B
                for i in range(numS):
                    A[0][:,i] = dirichlet.mean(A[0][:,i])
                    A[1][:,i] = dirichlet.mean(A[1][:,i])
                    A[2][:,i] = dirichlet.mean(A[2][:,i])
                    A[3][:,i] = dirichlet.mean(A[3][:,i])
                    A[4][:,i] = dirichlet.mean(A[4][:,i])
                    A[5][:,i] = dirichlet.mean(A[5][:,i])
                    A[6][:,i] = dirichlet.mean(A[6][:,i])
                    for j in range(numA):
                        B[0][:,i,j]=dirichlet.mean(B[0][:,i,j])

        rally_length_raw[trial][episode] = score+1

    env.close()

trial 0
trial 1
trial 2
trial 3
CPU times: total: 1h 10min 10s
Wall time: 1h 10min 12s


In [10]:
rally_length_1_raw = rally_length_raw[:, 0:int(episodes/4)]
rally_length_2_raw = rally_length_raw[:, int(episodes/4):episodes]

rally_length_1 = np.mean(rally_length_1_raw, axis=0)
rally_length_2 = np.mean(rally_length_2_raw, axis=0)
x = [rally_length_1, rally_length_2]

In [11]:
# Planning horizon 20

In [12]:
%%time

planning_horizon = 20
T = planning_horizon

rally_length_raw_20 = np.zeros((trials,episodes))

for trial in range(trials):
    print("trials", trial)
    
    EPS_VAL = 1e-16 #negligibleconstant
    # Likelhiood Dynamics
    a = utils.random_A_matrix(num_obs, num_states) * 0 + EPS_VAL
    A = utils.random_A_matrix(num_obs, num_states)

    # Transisition dynamics
    # Initialised as random becuase the agent need to learn the dynamics

    B = utils.random_B_matrix(num_states, num_controls)
    b = utils.random_B_matrix(num_states, num_controls) * 0 + EPS_VAL
    B[0].shape

    # Normalising A and B as probability distributions

    numS = num_states[0]
    numA = num_controls[0]

    for i in range(numS):
        A[0][:,i] = dirichlet.mean(a[0][:,i])
        A[1][:,i] = dirichlet.mean(a[1][:,i])
        A[2][:,i] = dirichlet.mean(a[2][:,i])
        A[3][:,i] = dirichlet.mean(a[3][:,i])
        A[4][:,i] = dirichlet.mean(a[4][:,i])
        A[5][:,i] = dirichlet.mean(a[5][:,i])
        A[6][:,i] = dirichlet.mean(a[6][:,i])
        for j in range(numA):
            B[0][:,i,j]=dirichlet.mean(b[0][:,i,j])

    # Prior preferences for biasing the generative model to control behaviour

    # The preferences are set uniform for all the hidden-states except the reward function
    C = utils.obj_array_uniform(num_obs)

    # Highest for the high-score and lowest for the lowscore
    C_score = np.array([-5.8, 0 , 1])
    # Normalising the prior preference
    C[6] = pymdp.maths.softmax(1*C_score)
    C[6]

    D = utils.obj_array_uniform([s1_size])
    
    for episode in range(episodes):
        # print("episode:", episode)

        state = env.reset()
        done = False
        score = 0

        t=0
        while not done:
            # env.render()
            if(t==0):
                qs = D
                qs_prev = qs

            Q_pi = action_dist(A[6], B, C[6], T=planning_horizon, sm_par=1)
            action = np.random.choice([0,1,2], size=None, replace=True, p=np.matmul(Q_pi[t],qs[0]))

            state, reward, done, info = env.step(action)
            score += reward

            observation = state_to_obs(state, reward)

            qs = pymdp.inference.update_posterior_states(A, observation, prior = D)
            if(t == 0):
                qs0 = qs

            pA = A
            pB = B
            pD = D

            # Learning
            A = pymdp.learning.update_obs_likelihood_dirichlet(pA, A, observation, qs, lr=1.0, modalities='all')

            actions = np.array([int(action)])
            B = pymdp.learning.update_state_likelihood_dirichlet(pB, B, actions, qs, qs_prev, lr=1.0, factors='all')

            if(t == 0):
                D = pymdp.learning.update_state_prior_dirichlet(pD, qs0, lr=1.0, factors='all')
                D[0] = pymdp.maths.softmax(D[0])

            t += 1
            qs_prev = qs

            if(t == T-1):
                t = 0
                #Normalising A and B
                for i in range(numS):
                    A[0][:,i] = dirichlet.mean(A[0][:,i])
                    A[1][:,i] = dirichlet.mean(A[1][:,i])
                    A[2][:,i] = dirichlet.mean(A[2][:,i])
                    A[3][:,i] = dirichlet.mean(A[3][:,i])
                    A[4][:,i] = dirichlet.mean(A[4][:,i])
                    A[5][:,i] = dirichlet.mean(A[5][:,i])
                    A[6][:,i] = dirichlet.mean(A[6][:,i])
                    for j in range(numA):
                        B[0][:,i,j]=dirichlet.mean(B[0][:,i,j])

        rally_length_raw_20[trial][episode] = score+1

    env.close()

trials 0
trials 1
trials 2
trials 3
CPU times: total: 6h 47min 21s
Wall time: 6h 47min 24s


In [13]:
rally_length_1_raw_20 = rally_length_raw_20[:, 0:int(episodes/4)]
rally_length_2_raw_20 = rally_length_raw_20[:, int(episodes/4):episodes]

rally_length_1_20 = np.mean(rally_length_1_raw_20, axis=0)
rally_length_2_20 = np.mean(rally_length_2_raw_20, axis=0)

x_20 = [rally_length_1_20, rally_length_2_20]

In [14]:
# Random agent

In [15]:
rally_length_raw_r = np.zeros((trials,episodes))

for trial in range(trials):
    print(trial)
    
    for episode in range(episodes):

        state = env.reset()
        done = False
        score = 0

        while not done:
            #env.render()
            action = env.action_space.sample()

            state, reward, done, info = env.step(action)
            score += reward

        rally_length_raw_r[trial][episode] = score+1

    env.close()

0
1
2
3


In [16]:
rally_length_1_raw_r = rally_length_raw_r[:, 0:int(episodes/4)]
rally_length_2_raw_r = rally_length_raw_r[:, int(episodes/4):episodes]

rally_length_1_r = np.mean(rally_length_1_raw_r, axis=0)
rally_length_2_r = np.mean(rally_length_2_raw_r, axis=0)
y = [rally_length_1_r, rally_length_2_r]

In [17]:
#Plotting

In [18]:
xy = [x[0], x[1], x_20[0], x_20[1], y[0], y[1]]

In [ ]:
from matplotlib import pyplot as plt

In [1]:
plt.boxplot(xy, showmeans=True, positions=[1,2,4,5,7,8])
plt.ylabel("Average rally length")
plt.xlabel("Act.Inf Agent(T=5), Act.Inf Agent(T=20), Random Agent")
plt.title("Game play")
plt.savefig('graph.png', dpi=500, bbox_inches='tight')

NameError: name 'plt' is not defined